## Transformer

Transformer in a deep learning context refers to the combination of mult-headed attention, combined with a couple of additional elements: layer normalization, feedforward network, and residual skip connections. Let's take each of these one-by-one:

### 1. Layer Normalization

> Unlike Batch Normalization and Instance Normalization, which applies scalar scale and bias for each entire channel/plane with the affine option, Layer Normalization applies per-element scale and bias with elementwise_affine ([torch docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)).

When you think of a single token's representation, it can get spikey. In an effort to smooth them out, we can use layer normalization. What does this mean? 

You subtract the mean and divide by the standard deviation. And you learn parameters, one or two for each embedding dimension. One if you disable the bias, two if you enable it.

$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$

Here's what it looks like in PyTorch and vanilla matrix multiplication.

In [63]:
import torch
import torch.nn as nn

batch_size = 2
context_size = 3
n_embd = 4
bias=False
# x = torch.tensor(range(batch_size * context_size * n_embd), dtype=torch.float).view(batch_size, context_size, n_embd)
x = torch.randint(high=4, size=(batch_size, context_size, n_embd), dtype=torch.float)
print(x)

layer_norm = nn.LayerNorm(n_embd, bias=bias)
print(layer_norm.weight)

output = layer_norm(x)
output

tensor([[[0., 1., 2., 2.],
         [1., 2., 0., 3.],
         [3., 3., 3., 2.]],

        [[3., 2., 2., 1.],
         [1., 3., 0., 1.],
         [3., 2., 0., 3.]]])
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)


tensor([[[-1.5075, -0.3015,  0.9045,  0.9045],
         [-0.4472,  0.4472, -1.3416,  1.3416],
         [ 0.5773,  0.5773,  0.5773, -1.7320]],

        [[ 1.4142,  0.0000,  0.0000, -1.4142],
         [-0.2294,  1.6059, -1.1471, -0.2294],
         [ 0.8165,  0.0000, -1.6330,  0.8165]]],
       grad_fn=<NativeLayerNormBackward0>)

In [65]:
eps = 1e-5
output2 = (x - torch.mean(x, dim=-1, keepdim=True)) / (torch.std(x, dim=-1, keepdim=True, unbiased=False) + eps)
output2

tensor([[[-1.5075, -0.3015,  0.9045,  0.9045],
         [-0.4472,  0.4472, -1.3416,  1.3416],
         [ 0.5773,  0.5773,  0.5773, -1.7320]],

        [[ 1.4142,  0.0000,  0.0000, -1.4142],
         [-0.2294,  1.6059, -1.1471, -0.2294],
         [ 0.8165,  0.0000, -1.6330,  0.8165]]])

In [66]:
gamma = torch.ones(n_embd) * 2
gamma

tensor([2., 2., 2., 2.])

In [68]:
output2 * gamma

tensor([[[-3.0151, -0.6030,  1.8090,  1.8090],
         [-0.8944,  0.8944, -2.6833,  2.6833],
         [ 1.1547,  1.1547,  1.1547, -3.4640]],

        [[ 2.8284,  0.0000,  0.0000, -2.8284],
         [-0.4588,  3.2118, -2.2941, -0.4588],
         [ 1.6330,  0.0000, -3.2660,  1.6330]]])

In [69]:
print(f"Outputs are equivalent: {torch.allclose(output, output2)}")

Outputs are equivalent: True
